In [1]:
# set theme
!jt -t oceans16

In [2]:
# this is to make the Jupyter notebook as wide as the screen (on the Mac at least)
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
%config InlineBackend.figure_format ='retina'

### Import needed modules

In [3]:
import BBP_RTQC
from BBP_RTQC_paths import *
from BBP_RTQC_global_vars import *
import subprocess
import numpy as np
import glob

import json #json manager
import re #regular expressions
from datetime import date
import os

In [4]:
# remove exising json file
fn_out = "BBP_RTQC_example_tests.json"
if os.path.exists(fn_out):
  os.remove(fn_out)

In [5]:
# these are the test functions, codes, and example files used to generate the json file
test_functions = {   "A": "BBP_Global_range_test",
                     "A2": "BBP_Global_range_test",
                     "B": "BBP_Noisy_profile_test",
                     "C": "BBP_High_Deep_Values_test",
                     "D": "BBP_Surface_hook_test", 
                     "E": "BBP_Missing_Data_test",
                     "F": "BBP_Negative_nonsurface_test",
                     "G": "BBP_Parking_hook_test",     
                     "H": "BBP_Stuck_Value_test"      
        }

# for each test a list of lists is given to add multiple examples. the first element of the list is the profile 
# used to extract the input data, the second element is a description of what this example is about and becomes 
# the "specifics" field in the final json file
egs = {  "A": [
                ["coriolis_BR7900591_240.nc", "Positive values"]
            ],
         "A2": [
                 ["coriolis_BR6903093_044.nc", "Negative values"]
             ],
         "B": [
                ["bodc_BR6901183_130.nc", "eg 1"], 
                ["aoml_BD5905108_073.nc", "eg 2"]
             ],
         "C": [
                 ["coriolis_BR6902827_363.nc", "eg 1"]
            ],
         "D": [
                 ["coriolis_BD6901482_097.nc", "eg 1"]
            ], 
         "E": [
                   ["coriolis_BR6902827_005.nc", "shallow profile with high deep values"], 
                   ["csiro_BD1901338_006.nc", "shallow profile with high deep values"], 
                   ["aoml_BD5906036_014.nc", "shallow profile due to missing data"],
                   ["coriolis_BR7900560_060.nc", "only one valid bin, but with data below 1000 dbars"], 
                   ["coriolis_BD7900591_077.nc", "only one bin"],
                   ["aoml_BD1902303_002.nc", "shallow profile, with good data below 200 dbar"], 
                   ["coriolis_BR7900586_001.nc", "shallow profile, no data deeper than 200 dbar"],
                   ["coriolis_BD6901580_120.nc", "missing data in some bins"],
                   ["csiro_BD5903955_554.nc", "no BBP data below 200 dbar"],
                   ["coriolis_BR6901772_087.nc", "no data at all in the upper 1000 dbar"],
              ],
         "F": [
                 ["coriolis_BD6901527_182.nc", "eg 1"]
             ],
         "G": [
                 ["coriolis_BD6901580_107.nc", "eg 1"]      
             ],
         "H": [
                 ["coriolis_BD6901580_107.nc", "eg 1"],
                 ["csiro_BR7900907_014.nc", "eg 2"]
             ]      
        }

In [6]:
def create_new_test_example(test, code, specs, test_function, PRES, COUNTS, BBP, BBPmf1, 
                            QC_FLAGS_IN, QC_FLAGS_OUT, maxPRES=None, PARK_PRES=None):
    
    new_eg_test = {"description": test, "code": code, "specifics": specs, "date": date.today().strftime("%Y-%m-%d"), "functions": [test_function],\
                    "input": {"PRES": PRES, "BBP": BBP, "BBPmf1": BBPmf1, "flags_in": QC_FLAGS_IN}, \
                    "output": {"flags_out": QC_FLAGS_OUT}  
                    }    
    if code=="G": # add maxPRES and PARK_PRES to inputs of G test
        new_eg_test['input'] = {"PRES": PRES, "BBP": BBP, "BBPmf1": BBPmf1, "flags_in": QC_FLAGS_IN,
                               "maxPRES": maxPRES, "PARK_PRES": PARK_PRES}
    if code=="H": # add COUNTS to inputs of H test
        new_eg_test['input'] = {"PRES": PRES, "COUNTS": COUNTS, "BBP": BBP, "flags_in": QC_FLAGS_IN}
        
        
        
    new_eg_test_str = json.dumps(new_eg_test, indent=2)   
    
    # format the string for better presentation
    new_eg_test_horizontal = new_eg_test_str.replace("\n", "")
    new_eg_test_horizontal = new_eg_test_horizontal.replace("  ", "")
    new_eg_test_horizontal = new_eg_test_horizontal.replace(",", ", ")

    return new_eg_test_horizontal

In [7]:
# initialise list of strings that will contain all the example-test inputs and outputs
all_eg_tests = []

In [8]:
# read data from example profile
def start_rd_4_json(fn_eg):

    # extract strings to find files
    tmp = fn_eg.split("_")
    iwmo = tmp[0]+'/'+tmp[1][2:]
    prof = tmp[-1][:3]#'080'
    QCstatus = tmp[1][0:2]#'BD'
    dac = iwmo.split('/')[0]
    wmo = iwmo.split('/')[-1]
    
    
    # read meta file
    [ds_config, SENSOR_MODEL, SENSOR_MAKER, SENSOR_SERIAL_NO, PLATFORM_TYPE, miss_no_float, dark, scale, khi] = BBP_RTQC.rd_WMOmeta(iwmo, VERBOSE=False)

    # list single profiles
    fn2glob = MAIN_DIR + "dac/" + iwmo + "/profiles/" + "B*" + iwmo.split("/")[-1] + "*_[0-9][0-9][0-9].nc"
    fn_single_profiles = np.sort(glob.glob(fn2glob))

    # select specific profile to QC
    fn_p = fn_single_profiles[0]
    fn_p = MAIN_DIR + "dac/" + iwmo + "/profiles/" + QCstatus+wmo+'_'+prof+'.nc'
    print(fn_p)
    # read BBP and PRES + other data from profile file
    [ PRES, BBP700, JULD, LAT, LON, BBP700mf1, miss_no_prof, PARK_PRES, maxPRES, innan, COUNTS] = BBP_RTQC.rd_BBP(fn_p, miss_no_float, ds_config)

    return iwmo, fn_p, PRES, BBP700, JULD, LAT, LON, BBP700mf1, miss_no_prof, PARK_PRES, maxPRES, innan, COUNTS

In [9]:
def ini_flags(BBP700):
    # initialise output array
    BBP700_QC_1st_failed_test = dict.fromkeys(tests.keys())
    for ikey in BBP700_QC_1st_failed_test.keys():
        BBP700_QC_1st_failed_test[ikey] = np.full(shape=BBP700.shape, fill_value='0')
    return BBP700_QC_1st_failed_test

In [10]:
# Global Range Test (A)
code = "A"
test = tests[code]
test_function = test_functions[code]
fn_eg = egs[code]

for i_eg in fn_eg:
    # read data from specific example profile
    [iwmo, fn_p, PRES, BBP700, JULD, LAT, LON, BBP700mf1, miss_no_prof, PARK_PRES, maxPRES, innan, COUNTS] = start_rd_4_json(i_eg[0])

    # create BBP700_QC_flags, BBP700_QC_1st_failed_test
    BBP700_QC_1st_failed_test = ini_flags(BBP700)

    # GLOBAL-RANGE TEST for BBP700
    QC_FLAGS_OUT, BBP700_QC_1st_failed_test = BBP_RTQC.BBP_Global_range_test(BBP700, BBP700mf1, PRES, 
                                                                             np.ones(BBP700.shape), BBP700_QC_1st_failed_test, 
                                                                             fn_p, VERBOSE=False, PLOT=False, SAVEPLOT=False)

    # create string to store results of example test
    specs = i_eg[1]
    tmp = create_new_test_example(test, code, specs, test_function, PRES.tolist(), COUNTS.tolist(),
                                  BBP700.tolist(), BBP700mf1.tolist(), 
                                  np.ones(BBP700.shape).tolist(), QC_FLAGS_OUT.tolist())

    # add example above to list to be saved
    all_eg_tests.append(tmp)

    del QC_FLAGS_OUT

/data/datasets/Argo/Original/dac/coriolis/7900591/profiles/BR7900591_240.nc


In [11]:
# Global Range Test: median-filtered negative values (A2)
code = "A2"
test = tests[code]
test_function = test_functions[code]
fn_eg = egs[code]

for i_eg in fn_eg:   
    # read data from specific example profile
    iwmo, fn_p, PRES, BBP700, JULD, LAT, LON, BBP700mf1, miss_no_prof, PARK_PRES, maxPRES, innan, COUNTS = start_rd_4_json(i_eg[0])

    # create BBP700_QC_flags, BBP700_QC_1st_failed_test
    BBP700_QC_1st_failed_test = ini_flags(BBP700)

    # GLOBAL-RANGE TEST for BBP700
    QC_FLAGS_OUT, BBP700_QC_1st_failed_test = BBP_RTQC.BBP_Global_range_test(BBP700, BBP700mf1, PRES, 
                                                                      np.ones(BBP700.shape), BBP700_QC_1st_failed_test, 
                                                                      fn_p, VERBOSE=False, PLOT=False, SAVEPLOT=False)

    # create string to store results of example test
    specs = i_eg[1]
    tmp = create_new_test_example(test, code, specs, test_function, PRES.tolist(), COUNTS.tolist(),
                                  BBP700.tolist(), BBP700mf1.tolist(), 
                                  np.ones(BBP700.shape).tolist(), QC_FLAGS_OUT.tolist())

    # add example above to list to be saved
    all_eg_tests.append(tmp)

    del QC_FLAGS_OUT

/data/datasets/Argo/Original/dac/coriolis/6903093/profiles/BR6903093_044.nc


In [12]:
# Noisy profile Test (B)
code = "B"
test = tests[code]
test_function = test_functions[code]
fn_eg = egs[code]

for i_eg in fn_eg:
    # read data from specific example profile
    iwmo, fn_p, PRES, BBP700, JULD, LAT, LON, BBP700mf1, miss_no_prof, PARK_PRES, maxPRES, innan, COUNTS = start_rd_4_json(i_eg[0])
    
    # create BBP700_QC_flags, BBP700_QC_1st_failed_test
    BBP700_QC_1st_failed_test = ini_flags(BBP700)

    # NOISY-PROFILE TEST for BBP700
    QC_FLAGS_OUT, BBP700_QC_1st_failed_test, res = BBP_RTQC.BBP_Noisy_profile_test(BBP700, BBP700mf1, PRES,
                                                                      np.ones(BBP700.shape), BBP700_QC_1st_failed_test, 
                                                                      fn_p, VERBOSE=False, PLOT=False, SAVEPLOT=False)

    # create string to store results of example test
    specs = i_eg[1]
    tmp = create_new_test_example(test, code, specs, test_function, PRES.tolist(), COUNTS.tolist(),
                                  BBP700.tolist(), BBP700mf1.tolist(), 
                                  np.ones(BBP700.shape).tolist(), QC_FLAGS_OUT.tolist())

    # add example above to list to be saved
    all_eg_tests.append(tmp)

    del QC_FLAGS_OUT

/data/datasets/Argo/Original/dac/bodc/6901183/profiles/BR6901183_130.nc
/data/datasets/Argo/Original/dac/aoml/5905108/profiles/BD5905108_073.nc


In [13]:
# High-Deep Value Test (C)
code = "C"
test = tests[code]
test_function = test_functions[code]
fn_eg = egs[code]

for i_eg in fn_eg:
    # read data from specific example profile
    iwmo, fn_p, PRES, BBP700, JULD, LAT, LON, BBP700mf1, miss_no_prof, PARK_PRES, maxPRES, innan, COUNTS = start_rd_4_json(i_eg[0])
    
    # create BBP700_QC_flags, BBP700_QC_1st_failed_test
    BBP700_QC_1st_failed_test = ini_flags(BBP700)

    # HIGH DEEP-VALUES TEST for BBP700
    QC_FLAGS_OUT, BBP700_QC_1st_failed_test = BBP_RTQC.BBP_High_Deep_Values_test(BBP700mf1, PRES, 
                                                                                 np.ones(BBP700.shape), BBP700_QC_1st_failed_test, 
                                                                                 fn_p, VERBOSE=False, PLOT=False, SAVEPLOT=False)

    # create string to store results of example test
    specs = i_eg[1]    
    tmp = create_new_test_example(test, code, specs, test_function, PRES.tolist(), COUNTS.tolist(),
                                  BBP700.tolist(), BBP700mf1.tolist(), 
                                  np.ones(BBP700.shape).tolist(), QC_FLAGS_OUT.tolist())

    # add example above to list to be saved
    all_eg_tests.append(tmp)

    del QC_FLAGS_OUT

/data/datasets/Argo/Original/dac/coriolis/6902827/profiles/BR6902827_363.nc


In [14]:
# SURFACE_HOOK TEST for BBP700
code = "D"
test = tests[code]
test_function = test_functions[code]
fn_eg = egs[code]

for i_eg in fn_eg:
    # read data from specific example profile
    iwmo, fn_p, PRES, BBP700, JULD, LAT, LON, BBP700mf1, miss_no_prof, PARK_PRES, maxPRES, innan, COUNTS = start_rd_4_json(i_eg[0])
    
    # create BBP700_QC_flags, BBP700_QC_1st_failed_test
    BBP700_QC_1st_failed_test = ini_flags(BBP700)

    # SURFACE_HOOK TEST for BBP700
    QC_FLAGS_OUT, BBP700_QC_1st_failed_test = BBP_RTQC.BBP_Surface_hook_test(BBP700, BBP700mf1, PRES, 
                                                                                 np.ones(BBP700.shape), BBP700_QC_1st_failed_test, 
                                                                                 fn_p, VERBOSE=False, PLOT=False, SAVEPLOT=False)

    # create string to store results of example test
    specs = i_eg[1]
    tmp = create_new_test_example(test, code, specs, test_function, PRES.tolist(), COUNTS.tolist(),
                                  BBP700.tolist(), BBP700mf1.tolist(), 
                                  np.ones(BBP700.shape).tolist(), QC_FLAGS_OUT.tolist())

    # add example above to list to be saved
    all_eg_tests.append(tmp)

    del QC_FLAGS_OUT

/data/datasets/Argo/Original/dac/coriolis/6901482/profiles/BD6901482_097.nc


In [15]:
# MISSING DATA TEST (E)
code = "E"
test = tests[code]
test_function = test_functions[code]
fn_eg = egs[code]

for i_eg in fn_eg:
    # read data from specific example profile
    iwmo, fn_p, PRES, BBP700, JULD, LAT, LON, BBP700mf1, miss_no_prof, PARK_PRES, maxPRES, innan, COUNTS = start_rd_4_json(i_eg[0])
    
    # create BBP700_QC_flags, BBP700_QC_1st_failed_test
    BBP700_QC_1st_failed_test = ini_flags(BBP700)

    # MISSING DATA TEST for BBP700
    QC_FLAGS_OUT, BBP700_QC_1st_failed_test = BBP_RTQC.BBP_Missing_Data_test(BBP700, PRES,
                                                                    np.ones(BBP700.shape), BBP700_QC_1st_failed_test, 
                                                                    fn_p, VERBOSE=False, PLOT=False, SAVEPLOT=False)

    # create string to store results of example test
    specs = i_eg[1]
    tmp = create_new_test_example(test, code, specs, test_function, PRES.tolist(), COUNTS.tolist(),
                                  BBP700.tolist(), BBP700mf1.tolist(), 
                                  np.ones(BBP700.shape).tolist(), QC_FLAGS_OUT.tolist())

    # add example above to list to be saved
    all_eg_tests.append(tmp)

    del QC_FLAGS_OUT

/data/datasets/Argo/Original/dac/coriolis/6902827/profiles/BR6902827_005.nc
/data/datasets/Argo/Original/dac/csiro/1901338/profiles/BD1901338_006.nc
/data/datasets/Argo/Original/dac/aoml/5906036/profiles/BD5906036_014.nc
/data/datasets/Argo/Original/dac/coriolis/7900560/profiles/BR7900560_060.nc
/data/datasets/Argo/Original/dac/coriolis/7900591/profiles/BD7900591_077.nc
/data/datasets/Argo/Original/dac/aoml/1902303/profiles/BD1902303_002.nc
/data/datasets/Argo/Original/dac/coriolis/7900586/profiles/BR7900586_001.nc
/data/datasets/Argo/Original/dac/coriolis/6901580/profiles/BD6901580_120.nc
/data/datasets/Argo/Original/dac/csiro/5903955/profiles/BD5903955_554.nc
/data/datasets/Argo/Original/dac/coriolis/6901772/profiles/BR6901772_087.nc


In [16]:
# NEGATIVE-NON-SURFACE TEST for BBP700
code = "F"
test = tests[code]
test_function = test_functions[code]
fn_eg = egs[code]

for i_eg in fn_eg:
    # read data from specific example profile
    iwmo, fn_p, PRES, BBP700, JULD, LAT, LON, BBP700mf1, miss_no_prof, PARK_PRES, maxPRES, innan, COUNTS = start_rd_4_json(i_eg[0])
    
    # create BBP700_QC_flags, BBP700_QC_1st_failed_test
    BBP700_QC_1st_failed_test = ini_flags(BBP700)

    # NEGATIVE-NON-SURFACE TEST for BBP700
    QC_FLAGS_OUT, BBP700_QC_1st_failed_test = BBP_RTQC.BBP_Negative_nonsurface_test(BBP700, PRES,
                                                                                 np.ones(BBP700.shape), BBP700_QC_1st_failed_test, 
                                                                                 fn_p, VERBOSE=False, PLOT=False, SAVEPLOT=False)

    # create string to store results of example test
    specs = i_eg[1]
    tmp = create_new_test_example(test, code, specs, test_function, PRES.tolist(), COUNTS.tolist(),
                                  BBP700.tolist(), BBP700mf1.tolist(), 
                                  np.ones(BBP700.shape).tolist(), QC_FLAGS_OUT.tolist())

    # add example above to list to be saved
    all_eg_tests.append(tmp)

    del QC_FLAGS_OUT

/data/datasets/Argo/Original/dac/coriolis/6901527/profiles/BD6901527_182.nc


In [17]:
# PARKING-HOOK TEST for BBP700
code = "G"
test = tests[code]
test_function = test_functions[code]
fn_eg = egs[code]

for i_eg in fn_eg:
    # read data from specific example profile
    iwmo, fn_p, PRES, BBP700, JULD, LAT, LON, BBP700mf1, miss_no_prof, PARK_PRES, maxPRES, innan, COUNTS = start_rd_4_json(i_eg[0])
    
    # create BBP700_QC_flags, BBP700_QC_1st_failed_test
    BBP700_QC_1st_failed_test = ini_flags(BBP700)

    # PARKING-HOOK TEST for BBP700
    QC_FLAGS_OUT, BBP700_QC_1st_failed_test = BBP_RTQC.BBP_Parking_hook_test(BBP700, BBP700mf1, PRES,
                                                                              maxPRES, PARK_PRES, 
                                                                                 np.ones(BBP700.shape), BBP700_QC_1st_failed_test, 
                                                                                 fn_p, VERBOSE=False, PLOT=False, SAVEPLOT=False)

    # create string to store results of example test
    specs = i_eg[1]
    tmp = create_new_test_example(test, code, specs, test_function, PRES.tolist(), COUNTS.tolist(),
                                  BBP700.tolist(), BBP700mf1.tolist(), 
                                  np.ones(BBP700.shape).tolist(), QC_FLAGS_OUT.tolist(),
                                  maxPRES.tolist(), PARK_PRES.tolist())

    # add example above to list to be saved
    all_eg_tests.append(tmp)

    del QC_FLAGS_OUT

/data/datasets/Argo/Original/dac/coriolis/6901580/profiles/BD6901580_107.nc


In [18]:
# STUCk-VALUE TEST for BBP700
code = "H"
test = tests[code]
test_function = test_functions[code]
fn_eg = egs[code]

for i_eg in fn_eg:
    # read data from specific example profile
    iwmo, fn_p, PRES, BBP700, JULD, LAT, LON, BBP700mf1, miss_no_prof, PARK_PRES, maxPRES, innan, COUNTS = start_rd_4_json(i_eg[0])
    
    if "BD6901580_107" in i_eg[0]:
        # example 1: set BBP700 to a constant value
        i1stgood = np.where(COUNTS>0)[0][0]
        BBP700[:] = BBP700[i1stgood]
        COUNTS[:] = COUNTS[i1stgood]
#         print('Stuck-value test: eg 1')


    # create BBP700_QC_flags, BBP700_QC_1st_failed_test
    BBP700_QC_1st_failed_test = ini_flags(BBP700)

    # PARKING-HOOK TEST for BBP700
    QC_FLAGS_OUT, BBP700_QC_1st_failed_test = BBP_RTQC.BBP_Stuck_Value_test(COUNTS, BBP700, PRES,
                                                                                 np.ones(BBP700.shape), BBP700_QC_1st_failed_test, 
                                                                                 fn_p, VERBOSE=False, PLOT=False, SAVEPLOT=False)

    # create string to store results of example test
    specs = i_eg[1]
    tmp = create_new_test_example(test, code, specs, test_function, PRES.tolist(), COUNTS.tolist(), BBP700.tolist(), 
                                  BBP700mf1.tolist(), 
                                  np.ones(BBP700.shape).tolist(), QC_FLAGS_OUT.tolist(),
                                  maxPRES.tolist(), PARK_PRES.tolist())

    # add example above to list to be saved
    all_eg_tests.append(tmp)

    del QC_FLAGS_OUT

/data/datasets/Argo/Original/dac/coriolis/6901580/profiles/BD6901580_107.nc
/data/datasets/Argo/Original/dac/csiro/7900907/profiles/BR7900907_014.nc


In [19]:
# convert list to string in json format
all_eg_tests_str = json.dumps(all_eg_tests, indent=2)

In [20]:
# Save json file
out_file = open(fn_out, "w")
out_file.write(all_eg_tests_str)
out_file.close()


In [21]:
all_eg_tests[-2]


'{"description": "Stuck Value", "code": "H", "specifics": "eg 1", "date": "2021-10-19", "functions": ["BBP_Stuck_Value_test"], "input": {"PRES": [0.10000000149011612, 0.0, 0.0, 0.10000000149011612, 0.0, 0.10000000149011612, 0.5, 0.4000000059604645, 0.5, 0.5, 0.4000000059604645, 0.800000011920929, 0.8999999761581421, 1.2000000476837158, 1.2999999523162842, 1.399999976158142, 1.600000023841858, 1.7999999523162842, 2.0999999046325684, 2.0999999046325684, 2.299999952316284, 2.5, 2.5999999046325684, 2.9000000953674316, 3.0, 3.200000047683716, 3.299999952316284, 3.5, 3.5999999046325684, 3.700000047683716, 4.099999904632568, 4.099999904632568, 4.099999904632568, 4.199999809265137, 4.300000190734863, 4.400000095367432, 4.400000095367432, 4.599999904632568, 4.599999904632568, 4.599999904632568, 4.599999904632568, 4.599999904632568, 4.699999809265137, 4.800000190734863, 4.800000190734863, 4.800000190734863, 4.800000190734863, 4.800000190734863, 4.800000190734863, 4.900000095367432, 4.90000009536